In [6]:
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Subset
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, Subset
import numpy as np
import random
from tqdm import tqdm
from sklearn.metrics import roc_auc_score

In [7]:
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split

# MNIST 데이터 불러오기
(X_train_full, y_train_full), (X_test_full, y_test_full) = mnist.load_data()

# 데이터 정규화 (0-255 값을 0-1 사이로)
X_train_full = X_train_full.astype('float32') / 255.0
X_test_full = X_test_full.astype('float32') / 255.0

#torch.transform / normalize // imagenet mean sd 

# 숫자 1과 7만 선택하는 마스크 생성
train_mask = np.isin(y_train_full, [0, 1, 7])
X_train, y_train = X_train_full[train_mask], y_train_full[train_mask]

test_mask = np.isin(y_test_full, [0, 1, 7])
X_test, y_test = X_test_full[test_mask], y_test_full[test_mask]

# 시드 고정 (예: 42로 고정)
np.random.seed(42)

# 2000개의 데이터를 무작위로 선택
num_samples = 300
indices = np.random.choice(len(X_train), num_samples, replace=False)
X_sampled, y_sampled = X_train[indices], y_train[indices]

# 2000개의 샘플에서 train/test 데이터 분할 (80% train, 20% test 비율로 나눔)
X_train, X_test, y_train, y_test = train_test_split(
    X_sampled, y_sampled, stratify=y_sampled,test_size=0.2, random_state=42
)

# y_train과 y_test에서 7을 2로 변환
y_train = np.where(y_train == 7, 2, y_train)
y_test = np.where(y_test == 7, 2, y_test)

# 결과
print(f"Training set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")


Training set size: 240
Test set size: 60


In [8]:
from keras.utils import to_categorical
y_train = to_categorical(y_train, num_classes=3)
y_test = to_categorical(y_test, num_classes=3)

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score, average_precision_score
import numpy as np
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split

# 하이퍼파라미터 설정
batch_size = 16
learning_rate = 0.001
num_epochs = 10

# X_train과 y_train이 올바르게 변환되고 동일한 첫 번째 차원을 가지는지 확인
X_train = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)  # 채널 차원 추가
y_train = torch.tensor(y_train, dtype=torch.long)  # 다중 클래스 분류를 위해 정수형 변환

X_test = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)
y_test = torch.tensor(y_test, dtype=torch.long)


# 데이터셋과 데이터 로더 정의
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# CNN 모델 정의
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 3)  # 클래스 개수에 맞게 설정
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, 64 * 7 * 7)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)  # CrossEntropyLoss는 로짓을 기대하므로 sigmoid 사용 안 함
        return x

In [11]:
# 모델 초기화
model = CNNModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [12]:
def multi_accuracy(preds, y):
    """
    배치별 정확도를 반환합니다 (예: 8/10 개 정답인 경우 0.8 반환)
    """
    pred_soft = torch.log_softmax(preds, dim=1)
    _, pred_index = torch.max(pred_soft, dim=1)
    correct_pred = (pred_index == y).float()  # y와 pred_index 모두 [batch_size] 형태로 맞춤
    acc = correct_pred.sum() / len(correct_pred)
    return acc

In [13]:
# 학습 루프
for epoch in tqdm(range(num_epochs)):
    optimizer.zero_grad()
    
    # 입력 텐서 변환 (추가 차원 제거하여 [batch_size, 1, 28, 28] 형태로 만듦)
    X_tensor = X_train.clone().detach().float()  # [batch_size, 1, 28, 28]
    predictions = model(X_tensor)  # [batch_size, num_classes] 형태의 로짓 출력

    # 레이블이 원-핫 인코딩된 경우 클래스 인덱스로 변환
    label = y_train.clone().detach().long()
    if label.dim() > 1 and label.size(1) > 1:  # 원-핫 인코딩 여부 확인
        label = torch.argmax(label, dim=1)  # 클래스 인덱스로 변환하여 [batch_size] 형태로 맞춤

    # `predictions`와 `label`의 형식 확인
    print(f"Predictions shape: {predictions.shape}, dtype: {predictions.dtype}")
    print(f"Label shape: {label.shape}, dtype: {label.dtype}")

    # 손실 및 정확도 계산
    loss = criterion(predictions, label)
    acc = multi_accuracy(predictions, label)
    print(f'에포크 {epoch + 1}/{num_epochs}, 정확도: {acc:.4f}, 손실: {loss.item():.4f}')
    
    # 역전파 및 옵티마이저 스텝
    loss.backward()
    optimizer.step()


  0%|          | 0/10 [00:00<?, ?it/s]/home/jaeyeob/anaconda3/envs/lstm/lib/python3.10/site-packages/torch/autograd/graph.py:825: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11060). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
100%|██████████| 10/10 [00:00<00:00, 49.28it/s]

Predictions shape: torch.Size([240, 3]), dtype: torch.float32
Label shape: torch.Size([240]), dtype: torch.int64
에포크 1/10, 정확도: 0.4083, 손실: 1.0979
Predictions shape: torch.Size([240, 3]), dtype: torch.float32
Label shape: torch.Size([240]), dtype: torch.int64
에포크 2/10, 정확도: 0.6708, 손실: 1.0100
Predictions shape: torch.Size([240, 3]), dtype: torch.float32
Label shape: torch.Size([240]), dtype: torch.int64
에포크 3/10, 정확도: 0.7083, 손실: 0.8956
Predictions shape: torch.Size([240, 3]), dtype: torch.float32
Label shape: torch.Size([240]), dtype: torch.int64
에포크 4/10, 정확도: 0.8417, 손실: 0.7544
Predictions shape: torch.Size([240, 3]), dtype: torch.float32
Label shape: torch.Size([240]), dtype: torch.int64
에포크 5/10, 정확도: 0.9292, 손실: 0.5997
Predictions shape: torch.Size([240, 3]), dtype: torch.float32
Label shape: torch.Size([240]), dtype: torch.int64
에포크 6/10, 정확도: 0.9625, 손실: 0.4509
Predictions shape: torch.Size([240, 3]), dtype: torch.float32
Label shape: torch.Size([240]), dtype: torch.int64
에포크 7

In [15]:
import torch
import numpy as np
from sklearn.metrics import roc_auc_score, precision_recall_fscore_support, average_precision_score

# 테스트 데이터 텐서로 변환
X_tensor = X_test.clone().detach().float()  # X_test 사용, [batch_size, 1, 28, 28] 형태 예상
predictions = model(X_tensor)  # [batch_size, num_classes] 형태의 출력

# 소프트맥스를 적용하여 확률로 변환
predictions = torch.softmax(predictions, dim=1)

# 레이블이 원-핫 인코딩된 경우 클래스 인덱스로 변환
label = y_test.clone().detach().long()  # y_test 사용
if label.dim() > 1 and label.size(1) > 1:  # 원-핫 인코딩 여부 확인
    label = torch.argmax(label, dim=1)  # 클래스 인덱스로 변환하여 [batch_size] 형태로 맞춤

# 손실 계산
loss = criterion(predictions, label)

# sklearn 지표 계산을 위해 predictions와 labels를 numpy 배열로 변환
preds_np = predictions.detach().numpy()  # 그래프에서 분리하여 numpy로 변환
labels_np = label.numpy()  # 레이블도 numpy로 변환

# 예측된 클래스 가져오기
predicted_classes = np.argmax(preds_np, axis=1)

# 정확도 계산
acc = multi_accuracy(predictions, label)

# AUROC (클래스별 one-vs-rest 방식) 계산
auroc = roc_auc_score(labels_np, preds_np, multi_class="ovr")

# labels_np가 원-핫 인코딩이 아닌 클래스 레이블 형식인지 확인
if labels_np.ndim > 1:
    labels_np = np.argmax(labels_np, axis=1)

# 정밀도, 재현율, F1 점수 계산 (다중 클래스의 경우 macro 평균)
precision, recall, f1, _ = precision_recall_fscore_support(labels_np, predicted_classes, average='macro')

# 각 클래스에 대한 AUPRC (정밀-재현 곡선 아래 영역) 계산
auprc = average_precision_score(labels_np, preds_np, average="macro")

# 결과 출력
print(f'\n정확도: {acc}\n')
print(f'AUROC: {auroc}\n')
print(f'정밀도 (macro): {precision}\n')
print(f'재현율 (macro): {recall}\n')
print(f'F1 점수 (macro): {f1}\n')
print(f'AUPRC (macro): {auprc}\n')



정확도: 0.9833333492279053

AUROC: 0.9995590828924161

정밀도 (macro): 0.9866666666666667

재현율 (macro): 0.9814814814814815

F1 점수 (macro): 0.983673469387755

AUPRC (macro): 0.9990253411306043

